In [1]:
import yfinance as yf
import pandas as pd
import os 

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')

In [3]:
starting_stock_data = pd.read_csv('data/clean/master_equity_data.csv')
starting_stock_data

,Tickers,Have_Data,Is_ETF,cik_str,title,Sector,Sector_Check
0,A,NaN,False,1090872.0,"AGILENT TECHNOLOGIES, INC.",Healthcare,True
1,AA,NaN,False,1675149.0,Alcoa Corp,Basic Materials,True
2,AAAU,NaN,True,1708646.0,Goldman Sachs Physical Gold ETF,NaN,True
3,AACB,NaN,False,2034334.0,Artius II Acquisition Inc.,Financial Services,True
4,AACG,NaN,False,1420529.0,ATA Creativity Global,Consumer Defensive,True
...,...,...,...,...,...,...,...
11293,ZWS,NaN,False,1439288.0,Zurn Elkay Water Solutions Corp,Industrials,True
11294,ZYBT,NaN,False,1975641.0,Zhengye Biotechnology Holding Ltd,Healthcare,True
11295,ZYME,NaN,False,1937653.0,Zymeworks Inc.,Healthcare,True
11296,ZYXI,NaN,False,846475.0,ZYNEX INC,Healthcare,True


Need to get data in chunks, and then from their check if thier are empty

In [ ]:
filled_sharpe = pd.read_csv('data/clean/sharpe_ratios.csv')
already_found_sharpes = filled_sharpe['Tickers'].drop_duplicates().to_list()


starting_stock_data['Have_Data'] = starting_stock_data['Tickers'].isin(already_found_sharpes)

In [22]:
unprocessed_stocks = starting_stock_data[starting_stock_data['Have_Data'] == False]['Tickers'].to_list()
unprocessed_stocks

['AACT-WT',
 'AADI',
 'AAM-WT',
 'ABLLW',
 'ABLVW',
 'ABPWW',
 'ABVEW',
 'ACCD',
 'ACHL',
 'ACHR-WT',
 'ACLEW',
 'ACONW',
 'ACRDF',
 'ADNWW',
 'ADSEW',
 'ADVWW',
 'ADZCF',
 'AE',
 'AEAEW',
 'AENTW',
 'AERTW',
 'AEVA-WT',
 'AFJKR',
 'AFRIW',
 'AGR',
 'AHL-PC',
 'AHPIQ',
 'AIDG',
 'AILEW',
 'AIMA',
 'AIMAW',
 'AIMDW',
 'AIRJW',
 'AISPW',
 'AITRR',
 'ALCYW',
 'ALDFW',
 'ALFUW',
 'ALIV',
 'ALMP',
 'ALRN',
 'ALSAR',
 'ALSAW',
 'ALTM',
 'ALTR',
 'ALUR-WT',
 'ALVOW',
 'ALVR',
 'AMBI-WT',
 'AMBP-WT',
 'AMODW',
 'AMPS',
 'AMPX-WT',
 'ANG-PA',
 'ANGHW',
 'ANNAW',
 'ANSCW',
 'AONCW',
 'AP-WT',
 'APCXW',
 'APTO',
 'APUS',
 'APXI',
 'AQU',
 'AQUNR',
 'ARBEW',
 'ARCH',
 'AREBW',
 'ARKOW',
 'ARLTF',
 'ARTH',
 'ASAI',
 'ASCRF',
 'ASCWF',
 'ASTLW',
 'ATCHW',
 'ATEKW',
 'ATIF',
 'ATIPW',
 'ATLEW',
 'ATMCR',
 'ATMCW',
 'ATMVR',
 'ATNFW',
 'ATSG',
 'AUROW',
 'AURS',
 'AUUDW',
 'AVPTW',
 'AVTE',
 'AWH',
 'AY',
 'AYR',
 'AYWWF',
 'AZPN',
 'BABY',
 'BACQR',
 'BAERW',
 'BALY-T',
 'BARK-WT',
 'BAYAR',
 'BAZNF'

In [49]:
from utils.helpers import divide_chunks

num_in_chunks = 15
# tickers = unprocessed_stocks.copy()

# tickers = already_found_sharpes 

tickers = ['AADI', 'ABLLW', 'A', 'AA', 'AAAU', 'AACB']	

ticker_chunks = list(divide_chunks(tickers, num_in_chunks))



for chunk in ticker_chunks:
    stock_data = yf.download(chunk, start='2025-06-01', end='2025-06-06')['Close']
    break

[*********************100%***********************]  6 of 6 completed

2 Failed downloads:
['ABLLW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-06-01 -> 2025-06-06)')
['AADI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-06-01 -> 2025-06-06) (Yahoo error = "No data found, symbol may be delisted")')


In [ ]:
import numpy as np


is_empty_array = [] # instance the list

# Assuming stock_data is your DataFrame
empty_cols = stock_data.columns[stock_data.isna().all()].values # find only cols (whcih are the tickers), that are empty

is_empty_array.extend(empty_cols) # extend them 

print(is_empty_array)

['AADI', 'ABLLW']
